In [1]:
import argparse
import torch
import os
import json
from tqdm import tqdm
import shortuuid
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import math
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer
import sys
import re
import gc


IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)


def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform


def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio


def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images


def load_image(image_file, input_size=448, max_num=12, use_dynamic=True):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    if use_dynamic:
        images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
        print("Use Dynamic")
    else:
        images = [image]
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values


def split_list(lst, n):
    """Split a list into n (roughly) equal-sized chunks"""
    chunk_size = math.ceil(len(lst) / n)  # integer division
    return [lst[i:i+chunk_size] for i in range(0, len(lst), chunk_size)]


def get_chunk(lst, n, k):
    chunks = split_list(lst, n)
    return chunks[k]


def load_json_jsonl(file_path):
    # Check the file extension and process accordingly
    if file_path.endswith(".jsonl"):
        # For JSONL files (line-delimited JSON)
        data = [json.loads(line) for line in open(file_path, "r")]
    elif file_path.endswith(".json"):
        # For standard JSON files
        with open(file_path, "r") as f:
            data = json.load(f)  # Load the entire JSON file as a single object
    else:
        raise ValueError(f"Unsupported file format: {file_path}")
    return data

/home/zilun/anaconda3/envs/imagerag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
answer_fpath = "/media/zilun/fanxiang4t/GRSM/ImageRAG_git/data/eval/answer_mme-realworld-lite-8B-detection-fit-union-obj.jsonl"
model_path = "/media/zilun/fanxiang4t/GRSM/ImageRAG_git/checkpoint/InternVL2_5-8B_lora_5param_dynamic_0-1000_obb2_fit_unionpatch_obj_merged"
image_root = "/media/zilun/wd-161/datasets/MME-RealWorld/rs_subset/remote_sensing"

In [3]:
answers = load_json_jsonl(answer_fpath)
example_answer = answers[0]
print(example_answer)
print(len(answers))

{'Question_id': 'perception/remote_sensing/position/2527', 'Image': 'dota_v2_dota_v2_dota_v2_P7036.png', 'Text': 'Where is the yellow crane in the picture?', 'Question Type': 'Multiple Choice', 'Answer choices': ['(A) In the upper right area of the picture', '(B) In the upper left area of the picture', '(C) In the lower right area of the picture', '(D) In the lower left area of the picture', "(E) This image doesn't feature the position."], 'Ground truth': 'A', 'Category': 'position', 'Subtask': 'Remote Sensing', 'Task': 'Perception', 'target_of_interest': 'The yellow crane in the picture.', 'visual_cue': [727.0, 371.0, 1000.0, 715.0], 'output': '(A)'}
148


In [4]:
# Model
model = AutoModel.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    load_in_8bit=True
).eval()
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True,
    use_fast=False
)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.60s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
generative_vlm_transform = build_transform(model.config.vision_config.image_size)
generative_vlm = model
generative_vlm_dynamic_preprocess = dynamic_preprocess
generative_vlm_tokenizer = tokenizer

In [6]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

def show_image_with_bbox(image_path, bboxes):
    """
    Display an image with bounding boxes in Jupyter Notebook at its original size.

    Args:
        image_path (str): Path to the image file.
        bboxes (list): List of bounding boxes, each represented as (x1, y1, x2, y2).
    """
    # 读取图片
    img = mpimg.imread(image_path)
    
    # 获取图像的原始尺寸
    img_height, img_width = img.shape[:2]
    
    # 创建图形和轴，设置图形大小为图像的原始尺寸
    dpi = 80  # 可以根据需要调整 DPI
    figsize = (img_width / dpi, img_height / dpi)
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    
    # 显示图片
    ax.imshow(img)

    if bboxes is not None:
        # 绘制每个边界框
        for bbox in bboxes:
            x1, y1, x2, y2 = bbox
            x1 = int(x1 / 1000 * img_width)
            y1 = int(y1 / 1000 * img_height)
            x2 = int(x2 / 1000 * img_width)
            y2 = int(y2 / 1000 * img_height)
            rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=5, edgecolor='blue', facecolor='none')
            ax.add_patch(rect)
    
    # 移除轴标记
    ax.axis('off')
    
    # 显示图形
    plt.show()

In [7]:

def prepare_input(image_root, model_config, line, test_prompt, use_dynamic=False):
    image_name = line["Image"]
    image_path = os.path.join(image_root, image_name)
    # pixel_values = load_image(
    #     image_path,
    #     input_size=model_config.vision_config.image_size,
    #     max_num=model_config.max_dynamic_patch,
    #     use_dynamic=use_dynamic
    # ).to(torch.bfloat16).cuda()

    choices = line['Answer choices']
    image_file = line["Image"]
    qs = line["Text"]
    choice_prompt = ' The choices are listed below: \n'
    for choice in choices:
        choice_prompt += choice + "\n"
    qs += choice_prompt + test_prompt + '\nThe best answer is:'

    images, tile_num_list = [], []

    image = Image.open(image_path).convert('RGB')
    w, h = image.size
    image_anyres = generative_vlm_dynamic_preprocess(
        image,
        max_num=generative_vlm.config.max_dynamic_patch,
        image_size=generative_vlm.config.vision_config.image_size,
        use_thumbnail=generative_vlm.config.use_thumbnail,
    )
    images += image_anyres
    pixel_values = [generative_vlm_transform(image) for image in images]
    pixel_values = torch.stack(pixel_values).to(torch.bfloat16).cuda()
    tile_num_list.append(len(image_anyres))
    # line["visual_cue"][1] -= 300
    predict_bbox = line["visual_cue"]
    
    if predict_bbox:
        gc.collect()
        torch.cuda.empty_cache()
        visual_cues = [predict_bbox]
        for object_coord in visual_cues:
            x1, y1, x2, y2 = object_coord
            x1 = int(x1 / 1000 * w)
            y1 = int(y1 / 1000 * h)
            x2 = int(x2 / 1000 * w)
            y2 = int(y2 / 1000 * h)
            image_crop = image.crop((x1, y1, x2, y2))
            images.append(image_crop)
            tile_num_list.append(1)
        pixel_values = [generative_vlm_transform(image) for image in images]
        pixel_values = torch.stack(pixel_values).to(torch.bfloat16).cuda()

        final_instruction = "<image>\n"
        final_instruction += "Additional information:\n"
        for i, bbox in enumerate(visual_cues):
            final_instruction += "Sub-patch {} at location <box>[[{:.2f}, {:.2f}, {:.2f}, {:.2f}]]</box>: <image>\n".format(
                i + 1, *bbox)
        final_instruction += "Look at the image and answer the question based on the provided additional information (location of sub-patches). \n"
        final_instruction += "Question: "
        final_instruction += qs
        num_patches_list = [pixel_values.size(0) - len(visual_cues), len(visual_cues)]
        # response = generative_vlm.chat(
        #     generative_vlm_tokenizer,
        #     pixel_values,
        #     final_instruction,
        #     generative_vlm_generation_config,
        #     num_patches_list=num_patches_list
        # )
    else:
        final_instruction = question_with_test_template
        num_patches_list = [pixel_values.size(0)]
        # response = generative_vlm.chat(
        #     generative_vlm_tokenizer,
        #     pixel_values,
        #     final_instruction,
        #     generative_vlm_generation_config,
        # )

    return image_path, pixel_values, num_patches_list, final_instruction, predict_bbox

In [8]:
def inference(model, tokenizer, image_tensors, prompt, generation_param, num_patches_list):
    prompts = [prompt]
    generation_config = dict(
        max_new_tokens=generation_param["max_new_tokens"],
        do_sample=True if generation_param["temperature"] > 0 else False,
        temperature=generation_param["temperature"],
        top_p=generation_param["top_p"],
        num_beams=generation_param["num_beams"],
    )
    with torch.inference_mode():
        response = generative_vlm.chat(
            generative_vlm_tokenizer,
            image_tensors,
            prompt,
            generation_config,
            num_patches_list=num_patches_list
        )
    return response

In [11]:
# test_prompt = "Select the best answer to the above multiple-choice question based on the image. Respond with only the letter (A, B, C, D, or E) of the correct option."
test_prompt = "Select the best answer to the above multiple-choice question based on the image."
# test_prompt = "<image>\nPlease provide the bounding box coordinate of the region this sentence describes: "
sample = 50
import random
random.shuffle(answers)

generation_param_dict = {
    "max_new_tokens": 128,
    "temperature": 0.5,
    "top_p": 0.99,
    "num_beams": 1,
}

In [14]:
# for answer in answers[:sample]:
#     image_path, image_tensors, num_patches_list, qs, predict_bbox = prepare_input(image_root, model.config, answer, test_prompt, use_dynamic=True)
#     print(qs)
#     print(image_tensors.shape, num_patches_list)
#     print(predict_bbox)
#     print("GT: {}\n".format(answer["Ground truth"]))
#     gc.collect()
#     torch.cuda.empty_cache()
#     response = inference(generative_vlm, generative_vlm_tokenizer, image_tensors, qs, generation_param_dict, num_patches_list)
#     print("Ouput: {}".format(response))
#     show_image_with_bbox(image_path, [predict_bbox])
#     print()
#     print()

In [4]:
def extract_characters_regex(s, choices):
    s = s.strip()
    answer_prefixes = [
        "The best answer is",
        "The correct answer is",
        "The answer is",
        "The answer",
        "The best option is"
        "The correct option is",
        "Best answer:"
        "Best option:",
    ]
    for answer_prefix in answer_prefixes:
        s = s.replace(answer_prefix, "")

    if len(s.split()) > 10 and not re.search("[ABCDE]", s):
        return ""
    matches = re.search(r'[ABCDE]', s)
    if matches is None:
        for choice in choices:
            if s.lower() in choice.lower():
                return choice[1]
        return ""
    return matches[0]